# Importing Libraries

In [1]:
import torch
#from torch import nn

#import numpy as np

#from sklearn import datasets

import matplotlib as mpl
#from matplotlib.colors import LinearSegmentedColormap
#from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt

#import reprlib
import sys

# Configuring Visualization Parameters

In [2]:
%matplotlib inline

In [3]:
XINHUI = "#7a7374"
XUEBAI = "#fffef9"
YINBAI = "#f1f0ed"
YINHUI = "#918072"

figure_size = (16, 9)

In [4]:
custom_params = {
    "axes.axisbelow": True,
    "axes.edgecolor": YINBAI,
    "axes.facecolor": XUEBAI,
    "axes.grid": True,
    "axes.labelcolor": XINHUI,
    "axes.spines.right": False,
    "axes.spines.top": False,
    "axes.titlecolor": XINHUI,
    "figure.edgecolor": YINBAI,
    "figure.facecolor": XUEBAI,
    "grid.alpha": .8,
    "grid.color": YINBAI,
    "grid.linestyle": "--",
    "grid.linewidth": 1.2,
    "legend.edgecolor": YINHUI,
    "patch.edgecolor": XUEBAI,
    "patch.force_edgecolor": True,
    "text.color": XINHUI,
    "xtick.color": YINHUI,
    "ytick.color": YINHUI,
}

mpl.rcParams.update(custom_params)

# Configuring Other Notebook Parameters

In [5]:
#reprlib_rules = reprlib.Repr()
#reprlib_rules.maxother = 250

# Pre-installing Custom Functions

In [6]:
sys.path.append("../")

In [7]:
from Modules import *

# Practicing in Stages

In [8]:
string = """

RegExr was created by gskinner.com.

Edit the Expression & Text to see matches. Roll over matches or the expression for details. PCRE & JavaScript flavors of RegEx are supported. Validate your expression with<>  < hgdhgjjkjbkb12 lkn dhcjvjkblkn> Tests mode <ggjk122 hlkkln 
jggkjbhkjbkb khgkgbkjbkj >.

The side bar includes a Cheatsheet, full Reference, and Help. You can also Save & Share with the Community and view patterns you create or favorite <dqddqs>in My Patterns.

Explore results with the Tools below. Replace & List output custom results. Details lists capture groups. Explain describes your expression in plain English.
"""

In [9]:
import re


regex_pattern = "\<[\s\S]*?\>"

if re.search(regex_pattern, string):
    while re.search(regex_pattern, string):
        
        positions = re.search(regex_pattern, string).span()
        print(positions)
        
        string = string[: positions[0]] + "⟨" + string[positions[0]+1:]
        string = string[: positions[1] - 1] + "⟩" + string[positions[1]:]
    print(string)
else:
    print("no")


(210, 212)
(214, 247)
(259, 302)
(453, 461)


RegExr was created by gskinner.com.

Edit the Expression & Text to see matches. Roll over matches or the expression for details. PCRE & JavaScript flavors of RegEx are supported. Validate your expression with⟨⟩  ⟨ hgdhgjjkjbkb12 lkn dhcjvjkblkn⟩ Tests mode ⟨ggjk122 hlkkln 
jggkjbhkjbkb khgkgbkjbkj ⟩.

The side bar includes a Cheatsheet, full Reference, and Help. You can also Save & Share with the Community and view patterns you create or favorite ⟨dqddqs⟩in My Patterns.

Explore results with the Tools below. Replace & List output custom results. Details lists capture groups. Explain describes your expression in plain English.



In [10]:
#string ="ffffff"


def sign_adjuster(string):
    regex_pattern = "\<[\s\S]*?\>"
    while re.search(regex_pattern, string):
        positions = re.search(regex_pattern, string).span()
        string = f"{string[:positions[0]]}⟨{string[positions[0] + 1:]}"
        string = f"{string[:positions[1] - 1]}⟩{string[positions[1]:]}"
    return string
    
a = sign_adjuster(string)
a

'\n\nRegExr was created by gskinner.com.\n\nEdit the Expression & Text to see matches. Roll over matches or the expression for details. PCRE & JavaScript flavors of RegEx are supported. Validate your expression with⟨⟩  ⟨ hgdhgjjkjbkb12 lkn dhcjvjkblkn⟩ Tests mode ⟨ggjk122 hlkkln \njggkjbhkjbkb khgkgbkjbkj ⟩.\n\nThe side bar includes a Cheatsheet, full Reference, and Help. You can also Save & Share with the Community and view patterns you create or favorite ⟨dqddqs⟩in My Patterns.\n\nExplore results with the Tools below. Replace & List output custom results. Details lists capture groups. Explain describes your expression in plain English.\n'

In [11]:
from torch import nn

class LR(nn.Module):

    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        pred = self.linear(x)
        return pred


torch.manual_seed(1)
model = LR(1, 1)

string = str(model.parameters())

In [12]:
tabulation = Form_Generator()
tabulation.heading_printer("Initialization of custom module class")

definitions = [
    """
class Model(nn.Module):

    def __init__(self, input_size, H1, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, H1)
        self.linear2 = nn.Linear(H1, output_size)

    def forward(self, x):
        x = torch.sigmoid(self.linear(x))
        x = torch.sigmoid(self.linear2(x))
        return x

    def predict(self, x):
        pred = self.forward(x)
        if pred >= 0.5:
            return 1
        else:
            return 0
"""
]
tabulation.definition_generator(definitions)

statements = ["""
torch.manual_seed(2)
model = Model(2, 4, 1)
"""]
tabulation.statement_generator(statements)

variables = ["model"]
values = [str(model)]
tabulation.variable_generator(variables, values)

expressions = ["model.parameters()", "list(model.parameters())", "aa"]
results = [str(model.parameters()), str("<"), str("<=")]
tabulation.expression_generator(expressions, results)

Initialization of custom module class

    +------------------------------------------------------+
    | Definition                                           |
    +------------------------------------------------------+
    | class Model(nn.Module):                              |
    |                                                      |
    |     def __init__(self, input_size, H1, output_size): |
    |         super().__init__()                           |
    |         self.linear = nn.Linear(input_size, H1)      |
    |         self.linear2 = nn.Linear(H1, output_size)    |
    |                                                      |
    |     def forward(self, x):                            |
    |         x = torch.sigmoid(self.linear(x))            |
    |         x = torch.sigmoid(self.linear2(x))           |
    |         return x                                     |
    |                                                      |
    |     def predict(self, x):               